### Column Transformer



- If we have to apply diff encoding technique to different features 



In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split 

import warnings
warnings.filterwarnings('ignore')

In [3]:
df= pd.read_csv("D:\\Sandesh\\100-days-of-machine-learning\\day28-column-transformer\\covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [11]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [17]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [23]:
X=df.iloc[:,0:5]
Y=df.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

### Method 1 - Lengthy Method 

- For all features write diff encoding then at end concatenate all of them.



In [15]:

# Fever has some missing value thus replace it with mean

from sklearn.impute import SimpleImputer
si=SimpleImputer()

X_train_fever= si.fit_transform(X_train[['fever']])
X_test_fever= si.transform(X_test[['fever']])


In [32]:

# Apply ordinal encoding ----> cough features

from sklearn.preprocessing import OrdinalEncoder

oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.transform(X_test[['cough']])



In [34]:

# Apply OneHotencoding ---> gender,city - Nominal Encoding
from sklearn.preprocessing import OneHotEncoder

oht=OneHotEncoder(dtype='int8',drop='first',sparse=False)
X_train_gender_city= oht.fit_transform(X_train[['gender','city']])
X_test_gender_city= oht.transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [60]:

# As all the features are in numpy array we have to convert the age in numpy 
# array and  then add all the features.

X_train_age=X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age=X_test.drop(columns=['gender','fever','cough','city']).values



In [61]:

X_train_tranformed=np.concatenate((X_train_age,
                                  X_train_gender_city,
                                  X_train_fever,X_train_cough
                                  ),axis=1)
                                                                    
X_test_tranformed=np.concatenate((X_test_age,
                                  X_test_gender_city,
                                  X_test_fever,X_test_cough
                                  ),axis=1)
                                

### Method 2 -  Simple Method 

- In single step , define all the encoding technique and features at once using column transformer.


In [65]:

# Define a column transformer and give all the features names
# and details about the encoding at once.

# 1 - give any name , 2 - encoding details , 3- feature name


from sklearn.compose import ColumnTransformer

transformed=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')



In [67]:
transformed.fit_transform(X_train).shape

(80, 7)

In [68]:
transformed.transform(X_test).shape

(20, 7)

### THE END 